In [ ]:
import tensorflow as tf
import numpy as np

from tensorflow import keras
from keras.datasets import boston_housing


#------------------- def methods -----------------------#


In [ ]:
# load data and analyze the type, rank and dimensionality for each rank 
(train_data, train_labels), (test_data, test_labels) = boston_housing.load_data()

print('Data types')
print('Train data and labels')
print(type(train_data))
print(type(train_labels))
print('\nTest data and labels')
print(type(test_data))
print(type(test_labels))

print('\nRank and dimensionality for each axis')
print('- train data:')
print(train_data.ndim)
print(train_data.shape)

print('- train labels:')
print(train_labels.ndim)
print(train_labels.shape)


print('\n- test data')
print(test_data.ndim)
print(test_data.shape)

print('- test labels:')
print(test_labels.ndim)
print(test_labels.shape)

In [ ]:
# having few data in training data involves that validation should be applied in a different way than done
# in binary and multiclassification, for instance using k-fold validation. Moreover the neural network shouldn't be so complex

In [ ]:
print('Train data content')
print(train_data[0], ' - label: ', train_labels[0])

print('\nTest data content')
print(test_data[0], ' - label: ', test_labels[0])

In [ ]:
# Data is in a good format (tensor) so we don't need to apply transformations
# However we need to apply feature normalization because the values printed above are expressed on different numerical scales
# train_data[0][2] << train_data[0][11]

mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

test_data -= mean
test_data /= std

In [ ]:
# We print again the data modified

print('Train data content')
print(train_data[0], ' - label: ', train_labels[0])

print('\nTest data content')
print(test_data[0], ' - label: ', test_labels[0])

In [ ]:
# we an build our NN considering the problem we are going to face

# model definition 
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(13,))) # 13 features, see the x dimensionality for the training_data tensor 
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1))

from keras import optimizers
          
model.compile(optimizer='rmsprop',
              loss='mse', 
              metrics=['mae']) 

In [ ]:
# Validation should be applied using k-fold approach because the data we could include in validation couldn't be
# generalize the data 
# ...

In [ ]:
fitStatistics = model.fit(train_data, train_labels, epochs=20, batch_size=16)

In [ ]:
print(model.evaluate(test_data, test_labels))